## Fichas de seguridad multilingües con IA

En este documento, se desarrollarán fichas de seguridad multilingues de productos de Amazon a escala utilizando la Inteligencia Artificial.

In [1]:
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor
from urllib.request import urlopen, Request
from dotenv import load_dotenv
import threading
import markdown
import pdfkit
import io
from datetime import datetime
import requests
from pprint import pprint

#### Funciones para comunicar con OpenAI

In [2]:
load_dotenv()
client = OpenAI()

def generate_response_stream(prev_prompt):
    response = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[
        {"role": "system", "content": "Eres un asistente util experto en crear documentos de seguridad para productos online."},
        {"role": "user", "content": str(prev_prompt)}],
    stream=True,
    max_tokens=1000)
    full_text = ""
    for chunk in response:
 
        if chunk.choices[0].delta.content is not None:
            stream = chunk.choices[0].delta.content
            print(stream, end="")
            full_text += stream
    
    return full_text

def generate_response_gpt3(prev_prompt):
    response = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": str(prev_prompt)}],
    max_tokens=1000
)
    selection = response.choices[0].message.content
    return selection

In [3]:
def get_product(product_url):
   payload = {
      'source': 'amazon',
      'user_agent_type': 'desktop',
      'render': 'html',
      'parse': True,
      'url': product_url}

   response = requests.request(
      'POST',
      'https://realtime.oxylabs.io/v1/queries',
      auth=('apoloisma', 'Javiisma2002'), 
      json=payload,
   )

   return response.json()

In [4]:
def parse_json(json_data):
    result = json_data["results"][0]
    asin = result["content"]["asin"]
    brand = result["content"]["brand"]
    product_name = result["content"]["product_name"]
    bulletpoints = result["content"]["bullet_points"]

    parsed_json = {"Asin": asin,
                   "Marca": brand,
                   "Nombre de Producto": product_name,
                   "Información relevante": bulletpoints}
    return parsed_json


#### Prompt main

In [6]:
prompt = ''' 

Basado en la siguiente información, genera una ficha de seguridad sobre el uso de este producto.

Son artículos como aspersores de riego, programadores de riego, o pistolas de riego. Fabricados en plástico y/o metal.

La ficha debe seguir la normativa GPSR (General Product Safety Regulation) impuesta por Amazon al igual que otro tipo de regulaciones.

La ficha, debe seguir la siguiente estructura:

- Descripción general del producto: Incluirá una breve descripción del programador de riego, destacando su propósito principal (por ejemplo, automatizar el riego de jardines).
- Materiales de fabricación: Se detallarán los materiales específicos utilizados, como tipos de plástico o metal, considerando su impacto en la seguridad y durabilidad.
- Instrucciones de uso: Explicación clara de cómo instalar y programar el dispositivo, incluyendo cualquier precaución para evitar daños al producto o al usuario.
- Mantenimiento y cuidados: Consejos para la limpieza, almacenamiento y manejo adecuado del programador de riego para asegurar su larga vida útil.
- Medidas de seguridad: Advertencias sobre los riesgos potenciales (como el riesgo de daños por agua o eléctricos) y cómo mitigarlos.
- Disposición al final de su vida útil: Información sobre cómo deshacerse o reciclar el producto de manera responsable.

Respondeme simplemente con la ficha de seguridad en formato 'Markdown'. No incluyas ``` al principio y final del markdown, solo dame el texto markdown.

Solo incluye los # para los titulos de la estructura mencionana previamente. 

Aquí información del producto. Ten en cuenta que el texto a sido extraído directamente de la página web, por lo que estará mal escrito:
'''

#### Descripcion de producto

In [7]:
product_description_test = """ 

Nombre: Aqua Control C4100 Programador de Riego para Jardín-con Pantalla LCD-Hasta 8 Arranques por día, gris

Acerca de este producto
Programador digital de riego automático para jardines, provisto con pantalla LCD, filtro metálico y 2 roscas para grifo de 3/4" y 1".
Automatiza el riego con la frecuencia y duración que queramos. Se pueden establecer hasta 8 programas al día, desde un total de 15 configuraciones diferentes.
Para facilitar su programación, cuenta con frontal extraíble y una pantalla LCD, muy intuitiva y fácilmente manejable."
Presión mínima de 1 bar y máxima de 7 para su correcto funcionamiento. Proporciona un caudal de unos 15 l/min.
Funciona con 1 pila alcalina de 9 voltios (no incluida). Por seguridad, cuando la batería está baja, la válvula se cierra automáticamente, evitando así posibles gastos de agua.

Programador de Riego con Pantalla LCD
Programador de grifo que permite establecer el riego de manera sencilla y rápida ajustando la duración y la frecuencia de riego a las necesidades de cada uno.

Podrás ajustar el temporizador para regar el jardín a primera hora de la mañana o al atardecer cuando el agua se evapora menos. Despreocúpate de regar durante tus vacaciones o incluso en tu día a día, minimizando las tareas de mantenimiento para una mayor comodidad.

Contenido del blíster: Programador C4100, adaptador de grifo 3/4" y 1", filtro de malla metálica e instrucciones en castellano y portugués.

Primer paso
Presiona la pestaña de la parte inferior de la unidad de control, inserta una batería de 9V (no incluida) y cierra correctamente para evitar filtraciones de agua en el circuito interno.

Segundo paso
Monta el programador de riego únicamente en posición vertical, enroscando primero el adaptador incluido en el grifo de salida y luego el temporizador en el adaptador.

Tercer paso
Programa el inicio y el fin del riego presionando el botón "Prog" y luego los botones "Día", "Hora" y "Min". Asegúrate de introducir la hora y el día actuales con el botón "Tiempo".

Resultado
El riego comenzará en el tiempo establecido. Presiona el botón "Man/Auto" cuando desees cambiar al riego manual, y el botón ESC para borrar los ajustes de programación.

"""

In [8]:
def return_html_language(language, brand):

    created_portuguese = "Criado em:"
    created_english = "Created on:"
    created_german = "Erstellt am:"
    created_french = "Créé le:"
    created_italian = "Creato il:"
    created_polish = "Utworzono:"
    created_russian = "Создано:"

    footer_html_portuguese = """
        <footer>
            <!-- Seu conteúdo do rodapé -->
            <p>Marca registrada: {brand} <br> Fabricante: Altadex S.A. <br> Endereço: Calle Perfumería 7. 28770. Colmenar Viejo. Madrid. Espanha. <br> www.altadex.com - info@altadex.com <br> Número de contato: +34 918457676 <br> </p>
        </footer>
    </body>
    </html>
    """.format(brand=brand)

    footer_html_english = """
            <footer>
                <!-- Your footer content -->
                <p>Trademark: {brand} <br> Manufacturer: Altadex S.A. <br> Address: Calle Perfumería 7. 28770. Colmenar Viejo. Madrid. Spain. <br> www.altadex.com - info@altadex.com <br> Contact number: +34 918457676 <br> </p>
            </footer>
        </body>
        </html>
        """.format(brand=brand)

    footer_html_german = """
            <footer>
                <!-- Ihr Footer-Inhalt -->
                <p>Warenzeichen:{brand} <br> Hersteller: Altadex S.A. <br> Adresse: Calle Perfumería 7. 28770. Colmenar Viejo. Madrid. Spanien. <br> www.altadex.com - info@altadex.com <br> Kontakt Nummer: +34 918457676 <br> </p>
            </footer>
        </body>
        </html> 
    """.format(brand=brand)

    footer_html_french = """
            <footer>
                <!-- Votre contenu de pied de page -->
                <p>Marque déposée: {brand} <br> Fabricant: Altadex S.A. <br> Adresse: Calle Perfumería 7. 28770. Colmenar Viejo. Madrid. Espagne. <br> www.altadex.com - info@altadex.com <br> Numéro de contact: +34 918457676 <br> </p>
            </footer>
        </body>
        </html>
        """.format(brand=brand)

    footer_html_italian = """
            <footer>
                <!-- Il tuo contenuto del piè di pagina -->
                <p>Marchio registrato: {brand} <br> Produttore: Altadex S.A. <br> Indirizzo: Calle Perfumería 7. 28770. Colmenar Viejo. Madrid. Spagna. <br> www.altadex.com - info@altadex.com <br> Numero di contatto: +34 918457676 <br> </p>
            </footer>
        </body>
        </html>
        """.format(brand=brand)

    footer_html_polish = """
            <footer>
                <!-- Twoja treść stopki -->
                <p>Znak towarowy: {brand} <br> Producent: Altadex S.A. <br> Adres: Calle Perfumería 7. 28770. Colmenar Viejo. Madrid. Hiszpania. <br> www.altadex.com - info@altadex.com <br> Numer kontaktowy: +34 918457676 <br> </p>
            </footer>
        </body>
        </html>
    """.format(brand=brand)

    footer_html_russian = """
            <footer>
                <!-- Ваше содержимое подвала -->
                <p>Торговая марка: {brand}<br> Производитель: Altadex S.A. <br> Адрес: Calle Perfumería 7. 28770. Colmenar Viejo. Madrid. España. <br> www.altadex.com - info@altadex.com <br> Контактный номер: +34 918457676 <br> </p>
            </footer>
        </body>
        </html>
    """.format(brand=brand)
    
    if language == "Portuguese":
        footer_html = footer_html_portuguese
        created = created_portuguese
        return footer_html, created

    if language == "English":
        footer_html = footer_html_english
        created = created_english
        return footer_html, created
    
    if language == "German":
        footer_html = footer_html_german
        created = created_german
        return footer_html, created

    if language == "French":
        footer_html = footer_html_french
        created = created_french
        return footer_html, created
    
    if language == "Italian":
        footer_html = footer_html_italian
        created = created_italian
        return footer_html, created
    
    if language == "Polish":
        footer_html = footer_html_polish
        created = created_polish
        return footer_html, created
    
    if language == "Russian":
        footer_html = footer_html_russian
        created = created_russian
        return footer_html, created
    
    footer_html = """
        <footer>
            <!-- Your footer content -->
            <p>Marca registrada: {brand} <br> Fabricante: Altadex S.A. <br> Dirección: Calle Perfumería 7. 28770. Colmenar Viejo. Madrid. España. <br> www.altadex.com - info@altadex.com <br> Número de contacto: +34 918457676 <br> </p>
        </footer>
    </body>
    </html>
    """.format(brand=brand)
    created = "Fecha de creación:"
    
    return footer_html, created

In [9]:
def markdown_to_pdf(markdown_text, output_filename, language, brand):
    
    current_date = datetime.now().strftime("%d/%m/%Y")

    footer_html, created = return_html_language(language, brand)
    
    header_html = f""" 
<!DOCTYPE html>
<html>
<head>
    <style>
        body, html {{
            margin: 0;
            padding: 0;
            height: 100%;
        }}
        .content {{
            min-height: 100%;
            position: relative;
            padding-bottom: 120px; /* Adjust based on footer height */
        }}
        img {{
            width: 30%;
            height: auto;
            padding-bottom: 50px;
        }}
        footer {{
            position: absolute;
            bottom: 0;
            width: 100%;
            height: 120px; /* Adjust based on your footer content */
            font-style: italic;
        }}
        .creation-date {{
            position: absolute;
            top: 0;
            right: 0;
            padding: 10px;
            font-style: italic;
            font-size:large;
        }}
    </style>
</head>
<body>
<div class="creation-date">{created} <br> {current_date}</div>
<img src="https://i.postimg.cc/DZwj5rnq/Logo-Altadex-1.png">
"""

    # Embedding the image at the top of the Markdown text
    markdown_text_with_image = markdown_text
    

    # Converting Markdown to HTML
    html_text = markdown.markdown(markdown_text_with_image)

    full_html_text = header_html + html_text + footer_html
    
    options = {
        'encoding': "UTF-8"
    }
    
    # Generating PDF from HTML
    pdfkit.from_string(full_html_text, "generated_documents/" + output_filename, options=options)

def persist_pdf(text, product, language, brand):
    markdown_to_pdf(text, f"{product}-{language}.pdf", language, brand)
    print("PDF generated successfully in", language + ".")

In [10]:
def generate_other_languages(text, product, language, brand):
    text = generate_response_gpt3('Translate the following text to ' + language + '.' + 'Only output the translated text in Markdown. Make sure it has the same format as the input you will be given. Only output the mardown, dont use ´´´ for delimeters or anything. ' + 'Here the text: ' + text)
    persist_pdf(text, product, language, brand)

In [11]:
def generate_all(product):
    print('Retrieving product from Oxylabs...')
    json_data = get_product(product)
    parsed_json_data = parse_json(json_data)
    
    print('\nRetrieved product! Passing to GPT.')
    product = parsed_json_data["Asin"]
    brand = parsed_json_data["Marca"]
    product_description = f""" 
    Nombre de Producto: {parsed_json_data["Nombre de Producto"]} \n
    Marca: {parsed_json_data["Marca"]}
    Informacion de producto: {parsed_json_data["Información relevante"]} \n
 """
    
    executor = ThreadPoolExecutor(10)
    full_text = generate_response_stream(prompt + product_description)
    persist_pdf(full_text, product, 'Spanish', brand)

    futures = []
    languages = ['Portuguese', 'English', 'German', 'French', 'Italian', 'Polish', 'Russian']
    for lang in languages:
        future = executor.submit(generate_other_languages, full_text, product, lang, brand)
        futures.append(future)

    print('\nLoading documents in other languages...')

In [12]:
full_text = """# Ficha de Seguridad para el Aqua Control C4100 Programador de Riego

## Descripción General del Producto
Este es un programador digital de riego automático para jardines con pantalla LCD, diseñado para programar y automatizar el riego de jardines con la frecuencia y duración que el usuario requiera.

## Materiales de Fabricación
El programador de riego está compuesto de plástico resistente y tiene un filtro metálico integrado. También cuenta con roscas para grifo metálicas de 3/4" y 1". Estos materiales ofrecen durabilidad y resistencia a la corrosión.

## Instrucciones de Uso
Primero, presione la pestaña de la parte inferior de la unidad de control e inserte una batería de 9V (no incluida). Asegúrese de cerrar correctamente para evitar filtraciones de agua en el circuito interno. Monta el programador de riego únicamente en posición vertical, enroscando primero el adaptador incluido en el grifo de salida y luego el temporizador en el adaptador. Para programar el inicio y el fin del riego, presione el botón "Prog", y luego los botones "Día", "Hora", y "Min". 

## Mantenimiento y Cuidados
Mantenga el programador de riego en un lugar seco cuando no esté en uso. Limpie regularmente la pantalla LCD con un paño suave para mantenerla legible. Recuerde reemplazar la batería a tiempo para evitar el cierre automático de la válvula.

## Medidas de Seguridad
Este dispositivo no debe ser manipulado por niños. Evite la instalación cerca de fuentes de calor o en lugares expuestos a temperaturas extremas. Evite sumergir el programador de agua completamente, ya que esto podría causar fallas electrónicas.

## Disposición al Final de su Vida Útil
Por favor, deshazte de este producto de acuerdo con las regulaciones locales de eliminación de desechos electrónicos. En muchos lugares, este producto se puede llevar a un punto de reciclaje de electrónicos.

Por favor, lea y siga todas las instrucciones y advertencias en el manual del propietario antes de instalar o usar este producto.PDF generated successfully in Spanish."""

In [13]:
def test_generate_all(full_text, product):
    executor = ThreadPoolExecutor(10)
    persist_pdf(full_text, product, 'Spanish', brand="Aqua Control")
    futures = []
    languages = ['Portuguese', 'English', 'German', 'French', 'Italian', 'Polish', 'Russian']
    #for lang in languages:
    #   future = executor.submit(generate_other_languages, full_text, product, lang)
    #   futures.append(future)

    print('\nLoading documents in other languages...')

### Demo Documentos de seguridad Amazon

In [14]:
product = 'https://www.amazon.es/AQUA-CONTROL-C4099O-Programador-Oscuro/dp/B07PDS3HB4?th=1'
generate_all(product)

Retrieving product from Oxylabs...

Retrieved product! Passing to GPT.
# Ficha de Seguridad del Producto

## Descripción general del producto

El **Aqua Control C4099O Programador de Riego** está diseñado para automatizar el sistema de riego en jardines de todo tipo, facilitando la programación del riego con múltiples frecuencias y duraciones. Este dispositivo asegura un uso eficiente del agua, ajustando los horarios de riego a las necesidades específicas del usuario y del jardín, ya sea cada ciertas horas o una vez por semana. Su principal beneficio es la comodidad y la eficacia en el mantenimiento de áreas verdes, garantizando una distribución adecuada del agua.

## Materiales de fabricación

El programador de riego Aqua Control C4099O está fabricado principalmente de **plásticos de alta calidad** y componentes **metálicos** duraderos. Los plásticos usados permiten resistencia a la intemperie y a la exposición prolongada al sol, mientras que los componentes metálicos son resistentes 

PDF generated successfully in English.
PDF generated successfully in Italian.
PDF generated successfully in Portuguese.
PDF generated successfully in French.
PDF generated successfully in Polish.
PDF generated successfully in German.
PDF generated successfully in Russian.
